In [3]:
import nltk
import pandas as pd
import re
import string
from pprint import pprint
from wordcloud import WordCloud
import gensim
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os

from nltk import PorterStemmer
from nltk import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')

pd.set_option('display.max_colwidth', 100)
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['from'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Akshay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Akshay\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Akshay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# clean text by removing punctuations and stopwords
def clean_text(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = re.split(r'\W+', text.lower())
    text = [word for word in tokens if word not in stopwords and len(word) > 3]
    return text

In [5]:
# stemming of given tokens
def stemming_text(tokenized_text):
    ps = PorterStemmer()
    text = [ps.stem(word) for word in tokenized_text]
    return text

In [6]:
# lemmetization of given tokens
def lemmetizing_text(tokenized_text):
    wn = WordNetLemmatizer()
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

In [7]:
# visualize wordnet from given list of sentences
def visualize_word_cloud(document_list):
    long_string = ','.join(list(document_list))
    wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
    wordcloud.generate(long_string)
    wordcloud.to_image().show()

In [8]:
# create bag of word for text
def create_bow_corpus(data_list):
    dictionary = gensim.corpora.Dictionary(data_list)
    corpus = [dictionary.doc2bow(text) for text in data_list]
    return corpus, dictionary

In [9]:
# create lda model
def create_lda_model(corpus, dictionary, num_topics):
    lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics)
    # pprint(lda_model.print_topics())
    return lda_model

In [10]:
# visualize lda topics
def visualize_topics_pyldavis(lda_model, corpus, id2word, num_topics):
    pyLDAvis.enable_notebook()

    LDAvis_data_filepath = os.path.join(r'lda_vis_files/ldavis_prepeared_' + str(num_topics))

    # # this is a bit time consuming - make the if statement True
    # # if you want to execute visualization prep yourself
    if 1 == 1:
        LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
        with open(LDAvis_data_filepath, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)
    # load the pre-prepared pyLDAvis data from disk
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, r'lda_vis_files/ldavis_prepeared_'+ str(num_topics) +'.html')
    return LDAvis_prepared


In [12]:
# read the data
data = pd.read_csv('data/papers.csv', nrows=100)
# keep only impotant columns
data = data.drop(columns=['id', 'event_type', 'pdf_name', 'year', 'title', 'abstract'])
data['cleaned_text'] = data['paper_text'].apply(lambda x: clean_text(x))
data['cleaned_text_lemma'] = data['cleaned_text'].apply(lambda x: lemmetizing_text(x))
# create corpus with bag of words
corpus, dictonary = create_bow_corpus(data['cleaned_text_lemma'])
# create lda model
lda_model = create_lda_model(corpus, dictonary, num_topics=10)
# visualize topic
plda = visualize_topics_pyldavis(lda_model, corpus, dictonary, 10)
plda

c:\Users\Akshay\Projects\natural-language-processing\venv\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.009001  0.003570       1        1  20.698530
8      0.005377 -0.008711       2        1  18.670454
1     -0.014828  0.000601       3        1  18.132459
6      0.004476  0.003869       4        1  15.053180
5     -0.003216 -0.000649       5        1   7.859322
2      0.000433 -0.001853       6        1   7.741675
7      0.001629  0.004653       7        1   4.563640
4     -0.002572  0.005600       8        1   3.671305
9      0.001606 -0.000466       9        1   1.984993
0     -0.001904 -0.006614      10        1   1.624442, topic_info=          Term         Freq        Total Category  logprob  loglift
478    network  1747.000000  1747.000000  Default  30.0000  30.0000
318   function   949.000000   949.000000  Default  29.0000  29.0000
459      model  1142.000000  1142.000000  Default  28.0000  28.0000
412   learning   990.000000   990.000000  Default  27.0000  27.0000
716     system   968.000000   968.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
731       time     7.428783   604.185702  Topic10  -5.8168  -0.2785
489     number     6.332758   485.023153  Topic10  -5.9764  -0.2185
515  parameter     6.658553   562.748188  Topic10  -5.9263  -0.3169
775     weight     6.450734   529.066255  Topic10  -5.9580  -0.2869
275    example     6.272735   490.585088  Topic10  -5.9859  -0.2394

[867 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
13324      1  0.159059  00000000000000
13324      2  0.159059  00000000000000
13324      3  0.159059  00000000000000
13324      4  0.318119  00000000000000
13324      6  0.159059  00000000000000
...      ...       ...             ...
11534      3  0.186910           zador
11534      4  0.186910           zador
11534      5  0.186910           zador
11534      8  0.186910           zador
13089      3  0.520340           zebra

[3259 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 2, 7, 6, 3, 8, 5, 10, 1])

In [14]:
# read the data
data = pd.read_csv('data/yelp.csv', nrows=100)
data = data.drop(columns=['business_id', 'date', 'review_id', 'stars', 'type', 'user_id'])
data['cleaned_text'] = data['text'].apply(lambda x: clean_text(x))
# lemmatize text
data['cleaned_text_lemma'] = data['cleaned_text'].apply(lambda x: lemmetizing_text(x))
# create corpus with bag of words
corpus, dictonary = create_bow_corpus(data['cleaned_text_lemma'])
# create lda model
lda_model = create_lda_model(corpus, dictonary, num_topics=5)
# visualize topic
plda = visualize_topics_pyldavis(lda_model, corpus, dictonary, 5)
plda

c:\Users\Akshay\Projects\natural-language-processing\venv\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.019551  0.022940       1        1  32.476832
2      0.030945 -0.026711       2        1  29.571405
1     -0.031787 -0.019267       3        1  16.586049
3     -0.010233  0.022230       4        1  13.031268
0     -0.008475  0.000808       5        1   8.334446, topic_info=         Term       Freq      Total Category  logprob  loglift
24       food  36.000000  36.000000  Default  30.0000  30.0000
281   service  21.000000  21.000000  Default  29.0000  29.0000
30       like  28.000000  28.000000  Default  28.0000  28.0000
503  friendly  17.000000  17.000000  Default  27.0000  27.0000
157      park   2.000000   2.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
13       came   1.176828  14.054330   Topic5  -6.0837   0.0047
105    review   1.192194  15.235598   Topic5  -6.0708  -0.0631
304      want   1.157442  13.129181   Topic5  -6.1004   0.0561
291      time   1.094741  26.917893   Topic5  -6.1560  -0.7175
80     friend   1.068430  15.665604   Topic5  -6.1804  -0.2005

[398 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2293      4  0.775361        48th
1         1  0.186867  absolutely
1         2  0.560601  absolutely
1         3  0.186867  absolutely
789       2  0.661018    abstract
...     ...       ...         ...
130       5  0.079548       would
1725      4  0.753303   yardhouse
902       1  0.164558        year
902       2  0.493674        year
902       3  0.164558        year

[645 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 4, 1])